# Sequence Alignment

We'll write a tool to align two sequences using the Needleman-Wunsch algorithm.

We start by defining our sequences, along with our scoring parameters.

In [ ]:
seq1 = 'THISLINE'
seq2 = 'ISALIGNED'

match = 1
mismatch = -1
gap = -2

We'll use a `numpy` matrix for our score matrix. The size of this matrix needs to be the length of each sequence, _plus one_. We can create a 2-dimensional `numpy` matrix in the same way we created a 1-dimensional `numpy` array, using `zeros`. Instead of a single length, we use a tuple to indicate the dimensions of the matrix. Notice that the first dimension, corresponding to `seq1`, is the row number in this matrix and the second dimension, corresponding to `seq2`, is the column number. You can tell because `seq2` is one position longer than `seq1`, and likewise, there's one more column than row.

In [ ]:
import numpy as np


We'll start by initializing the top and left-hand edges of the matrix. 

First, the very top-left corner is zero.

The top of the matrix corresponds to gaps at the start in the aligned `seq1`, where the column / j number goes up but the row / i number stays 0.

Because these entries are at the edge of the score matrix, the only possible way to make an alignment is to come from the previous cell on the top edge and add a gap. This means that the score formula is
```
score of (0, j) = score of (0, j-1) + gap
```

In [ ]:

print(scores)

Now, we'll handle the left-hand edge of the matrix in the same way.

In [ ]:

print(scores)

For the rest of the table, we have three options:
* gap in sequence 1, so look in the cell with the same position in `seq1` (same i) but one earlier position in `seq2` (j - 1) and add a gap penalty
* gap in sequence 2, so look in the cell with one earlier position in `seq1` (i - 1) and the same position in `seq2` (same j)
* align the nucleotides from sequence 1 and sequence 2, which could be a match or a mismatch. We look in the cell above and to the left (i - 1 and j - 1), and then add the (mis)match score. One tricky point: when we're in row 1, we're looking back to `seq1` position 0, and so forth. 

We'll compute each of these three scores -- `gap1_score`, `gap2_score`, and `pair_score` -- and then choose the best of these three options.

We'll loop over each position in the matrix and fill it in using the best score. When computing the score for a cell, we need to be sure that every other cell to the left and/or above is filled in already. Looping over rows from `i=1` to the end, and then within each row looping over columns from `j=1` to the end, is a safe way to do this.

In [ ]:

print(scores)

## Alignment score statistics

We can use the code from the cells above and turn them into a function that takes two sequences as input and computes an alignment score.

In [ ]:
def align_score(s1, s2):
    scores = np.zeros((len(s1)+1, len(s2)+1))
    scores[0,0] = 0
    for j in range(1, len(s2)+1):
        scores[0,j] = scores[0, j-1] + gap
    for i in range(1, len(s1)+1):
        scores[i,0] = scores[i-1,0] + gap
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            gap1_score = scores[i][j-1] + gap
            gap2_score = scores[i-1][j] + gap
            if s1[i-1] == s2[j-1]:
                pair_score = scores[i-1][j-1] + match
            else:
                pair_score = scores[i-1][j-1] + mismatch
            if gap1_score > gap2_score and gap1_score > pair_score:
                scores[i][j] = gap1_score
            elif gap2_score > gap1_score and gap2_score > pair_score:
                scores[i][j] = gap2_score
            else:
                scores[i][j] = pair_score
    return(scores[len(s1),len(s2)])
align_score('THISLINE', 'ISALIGNED')

Now we can generate _random_ sequences and test their alignment score against a query sequence.

In [ ]:
query = 'ACGTACGTACGTACGTACGT'


We can plot a histogram of the scores

In [ ]:
import matplotlib.pyplot as plt


We can pick out the highest-scoring sequences

Biopython has pair-wise alignent built in. We can use the `PairwiseAligner` from the `Bio.Align` module to run the same computation that we carried out. First, we need to set up the right parameters for the aligner.

In [ ]:
!pip3 install biopython
from Bio import Align
aligner = Align.PairwiseAligner()
print(aligner)
aligner.score("THISLINE", "ISALIGNED")

The biopython aligner runs much faster than the tool we wrote here, so we can generate a lot more samples

Based on these samples, we can get a get a better estimate of rare, high-scoring alignments.

You can also see an informative web tool that computes alignment matrices here:
[Online Needleman-Wunsch tool](http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Needleman-Wunsch)

## Bonus: Computing the optimal alignment

Usually, we want to know the actual alignment between sequences, not just the score. To do this, we need to
* keep track of the "arrows" in our matrix -- where did the score "come from"?
* walk backwards through the matrix

In [ ]:
def align(s1, s2):
    # Initialize a matrix of scores and "back steps"
    scores = np.zeros((len(s1)+1, len(s2)+1))
    paths = np.full((len(s1)+1, len(s2)+1), 'NONE')
    
    scores[0,0] = 0
    paths[0,0] = 'INIT'
    
    print(scores)
    print(paths)
    for j in range(1, len(s2)+1):
        scores[0,j] = scores[0, j-1] + gap
        paths[0,j] = 'GAP1'
    print(scores)
    print(paths)

    for i in range(1, len(s1)+1):
        scores[i,0] = scores[i-1,0] + gap
        paths[i,0] = 'GAP2'
    print(scores)
    print(paths)
    
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            gap1_score = scores[i][j-1] + gap
            gap2_score = scores[i-1][j] + gap
            if s1[i-1] == s2[j-1]:
                pair_score = scores[i-1][j-1] + match
            else:
                pair_score = scores[i-1][j-1] + mismatch
            if gap1_score > gap2_score and gap1_score > pair_score:
                scores[i][j] = gap1_score
                paths[i][j] = 'GAP1'
            elif gap2_score > gap1_score and gap2_score > pair_score:
                scores[i][j] = gap2_score
                paths[i][j] = 'GAP2'
            else:
                scores[i][j] = pair_score
                paths[i][j] = 'PAIR'

    print(scores)
    print(paths)
    
    align1 = ''
    align2 = ''
    i = len(s1)
    j = len(s2)
    while i > 0 or j > 0:
        position = paths[i, j]
        print("From", (i, j), "path", position)
        if position == 'GAP1':
            j = j - 1
            align1 = '-' + align1
            align2 = s2[j] + align2
        elif position == 'GAP2':
            i = i - 1
            align1 = s1[i] + align1
            align2 = '-' + align2
        elif position == 'PAIR':
            i = i - 1
            j = j - 1
            align1 = s1[i] + align1
            align2 = s2[j] + align2
        else:
            print("Unknown position", position)
            exit(1)
    return (scores[len(s1),len(s2)], align1, align2)

align('THISLINE', 'ISALIGNED')